# Without Fine tuning
Only adding the entity to the ruler

In [ ]:
import spacy

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
ruler = nlp.add_pipe('entity_ruler')

In [ ]:
patterns = [
    #Menu Items
    {"label":"BEVERAGE","pattern":"cappuccino"},
    {"label":"BEVERAGE","pattern":"cappuccinos"},
    {"label":"BEVERAGE","pattern":"latte"},
    {"label":"BEVERAGE","pattern":"lattes"},
    #Sizes
    {"label":"SIZE","pattern":"small"},
    {"label":"SIZE","pattern":"medium"},
    {"label":"SIZE","pattern":"large"},
    #Cardinals
    {"label":"CARDINAL","pattern":"a"},
]

In [ ]:
ruler.add_patterns(patterns)

In [ ]:
# Testing before fine tuning
sentence = "I'd like a medium Cappuccino with extra foam and two small Lattes."
doc = nlp(sentence.lower())
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities [('a', 'CARDINAL'), ('medium', 'SIZE'), ('cappuccino', 'BEVERAGE'), ('two', 'CARDINAL'), ('small', 'SIZE'), ('lattes', 'BEVERAGE')]


In [ ]:
# More complex sentence
sentence = "I'd like a large Cappuccino with extra foam, two medium Lattes and a small cappuccino."
doc = nlp(sentence.lower())
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities [('a', 'CARDINAL'), ('large', 'SIZE'), ('cappuccino', 'BEVERAGE'), ('two', 'CARDINAL'), ('medium', 'SIZE'), ('lattes', 'BEVERAGE'), ('a', 'CARDINAL'), ('small', 'SIZE'), ('cappuccino', 'BEVERAGE')]


In [ ]:
# More complex sentence
sentence = "I'd like a large Cappuccino and two small cappuccinos."
doc = nlp(sentence.lower())
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities [('a', 'CARDINAL'), ('large', 'SIZE'), ('cappuccino', 'BEVERAGE'), ('two', 'CARDINAL'), ('small', 'SIZE'), ('cappuccinos', 'BEVERAGE')]


In [ ]:
# order_dict = {'Item':[],
#               'Size':[],
#               'Quantity':[]}

# for ent in doc.ents:

In [ ]:
# Create the function for setting up Spacy nlp object
def setup_spacy_ner(pattern):
  import spacy
  nlp = spacy.load("en_core_web_sm")
  ruler = nlp.add_pipe('entity_ruler')
  ruler.add_patterns(patterns)
  return nlp

In [ ]:
# Define the patterns according to Menu

patterns = [
    #Menu Items
    {"label":"BEVERAGE","pattern":"cappuccino"},
    {"label":"BEVERAGE","pattern":"cappuccinos"},
    {"label":"BEVERAGE","pattern":"latte"},
    {"label":"BEVERAGE","pattern":"lattes"},
    #Sizes
    {"label":"SIZE","pattern":"small"},
    {"label":"SIZE","pattern":"medium"},
    {"label":"SIZE","pattern":"large"},
    #Cardinals
    {"label":"CARDINAL","pattern":"a"},
]

# Initiate nlp object of Spacy with the patterns
nlp = setup_spacy_ner(patterns)

In [ ]:
# Testing the nlp object's ner function

sentence = "I'd like a large Cappuccino with extra foam, two medium Lattes and a small cappuccino."
doc = nlp(sentence.lower())
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities [('a', 'CARDINAL'), ('large', 'SIZE'), ('cappuccino', 'BEVERAGE'), ('two', 'CARDINAL'), ('medium', 'SIZE'), ('lattes', 'BEVERAGE'), ('a', 'CARDINAL'), ('small', 'SIZE'), ('cappuccino', 'BEVERAGE')]


In [ ]:
# Create a function to log in the order details
order_dict = {'Item':[],
              'Size':[],
              'Quantity':[]}

def insert_order(sentence):
  doc = nlp(sentence.lower())
  for i,ent in enumerate(doc.ents):
    if ent.label_ == 'BEVERAGE':
      order_dict['Item'].append(ent.text)
    elif ent.label_ == 'SIZE':
      order_dict['Size'].append(ent.text)
    elif ent.label_ == 'CARDINAL':
      if ent.text == 'a' or ent.text == 'an':
        order_dict['Quantity'].append('One')
      else:
        order_dict['Quantity'].append(ent.text)

  return order_dict

In [ ]:
insert_order("I'd like a Cappuccino with extra foam, two medium Lattes and a small cappuccino.")

{'Item': ['cappuccino', 'lattes', 'cappuccino'],
 'Size': ['medium', 'small'],
 'Quantity': ['One', 'two', 'One']}

# Fine Tuning (100 sample data)

In [ ]:
# Load pre-existing spacy model
import spacy
nlp=spacy.load('en_core_web_sm')


In [ ]:
# Adding the entity to the ruler before fine tuning
ruler = nlp.add_pipe('entity_ruler')

patterns = [
    {"label":"BEVERAGE","pattern":"Cappuccino"}
]

ruler.add_patterns(patterns)

In [ ]:
# Testing before fine tuning
doc = nlp("I'd like a large Cappuccino with extra foam.")
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities [('Cappuccino', 'PERSON')]


We can see that it is incorrectly categorizing "Cappuccino" as a "PERSON"

In [ ]:
# Getting the pipeline component
ner=nlp.get_pipe("ner")

In [ ]:
def generate_entity_output(sentence, word, entity_name):
    start_index = sentence.index(word)
    end_index = start_index + len(word) - 1
    output = {
        "entities": [
            (start_index, end_index, entity_name)
        ]
    }
    return (sentence, output)

In [ ]:
with open('custom_dataset_Cappuccino.txt', encoding='UTF-8') as file:
    # lines = [line.rstrip() for line in file]
    lines = [line.strip('"').rstrip().rstrip('"') for line in file]

In [ ]:
train_set = []

for line in lines:
  train_set.append(generate_entity_output(line,"Cappuccino","BEVERAGE"))

In [ ]:
from spacy import training
# Adding labels to the `ner`

for _, annotations in train_set:
  for ent in annotations.get("entities"):
    ner.add_label(ent[2])

In [ ]:
# Disable pipeline components you dont need to change
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [ ]:
train_set[0]

("I'll have a small Cappuccino, please.", {'entities': [(18, 27, 'BEVERAGE')]})

In [ ]:
# Import requirements
import random
from spacy.util import minibatch, compounding
from pathlib import Path
from spacy.training import Example

# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):

  # Training for 30 iterations
  for iteration in range(100):

    # shuufling examples  before every iteration
    random.shuffle(train_set)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = minibatch(train_set, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        example = []
        # Update the model with iterating each text
        for i in range(len(texts)):
            doc = nlp.make_doc(texts[i])
            example.append(Example.from_dict(doc, annotations[i]))
        
        # Update the model
        nlp.update(example, drop=0.5, losses=losses)

        print("Losses", losses)

In [ ]:
# Testing the model
doc = nlp("I'd like a large Cappuccino with extra foam.")
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities [('Cappuccino', 'BEVERAGE')]


# Fine Tune (50 sample data)

In [ ]:
# Load pre-existing spacy model
import spacy
nlp=spacy.load('en_core_web_sm')

In [ ]:
# Adding the entity to the ruler before fine tuning
ruler = nlp.add_pipe('entity_ruler')

patterns = [
    {"label":"BEVERAGE","pattern":"Cappuccino"}
]

ruler.add_patterns(patterns)

In [ ]:
# Testing before fine tuning
doc = nlp("I'd like a large Cappuccino with extra foam.")
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities [('Cappuccino', 'PERSON')]


We can see that it is incorrectly categorizing "Cappuccino" as a "PERSON"

In [ ]:
# Getting the pipeline component
ner=nlp.get_pipe("ner")

In [ ]:
def generate_entity_output(sentence, word, entity_name):
    start_index = sentence.index(word)
    end_index = start_index + len(word) - 1
    output = {
        "entities": [
            (start_index, end_index, entity_name)
        ]
    }
    return (sentence, output)

In [ ]:
with open('custom_dataset_Cappuccino.txt', encoding='UTF-8') as file:
    # lines = [line.rstrip() for line in file]
    lines = [line.strip('"').rstrip().rstrip('"') for line in file]

In [ ]:
train_set = []

for line in lines[:10]:
  train_set.append(generate_entity_output(line,"Cappuccino","BEVERAGE"))

In [ ]:
len(train_set)

10

In [ ]:
from spacy import training
# Adding labels to the `ner`

for _, annotations in train_set:
  for ent in annotations.get("entities"):
    ner.add_label(ent[2])

In [ ]:
# Disable pipeline components you dont need to change
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [ ]:
train_set[0:10]

[('Can I order a small Cappuccino with chocolate sprinkles?',
  {'entities': [(20, 29, 'BEVERAGE')]}),
 ('Could I get a medium Cappuccino with almond milk?',
  {'entities': [(21, 30, 'BEVERAGE')]}),
 ('Can I get a medium Cappuccino to go?',
  {'entities': [(19, 28, 'BEVERAGE')]}),
 ('Could I get a small Cappuccino with a sprinkle of cocoa powder?',
  {'entities': [(20, 29, 'BEVERAGE')]}),
 ("I'd like a medium Cappuccino with a dash of vanilla syrup.",
  {'entities': [(18, 27, 'BEVERAGE')]}),
 ('May I have a large Cappuccino with whipped cream, please?',
  {'entities': [(19, 28, 'BEVERAGE')]}),
 ("I'll take a large Cappuccino with a shot of caramel.",
  {'entities': [(18, 27, 'BEVERAGE')]}),
 ("I'd like a large Cappuccino with extra foam.",
  {'entities': [(17, 26, 'BEVERAGE')]}),
 ('May I have a small Cappuccino with cinnamon on top?',
  {'entities': [(19, 28, 'BEVERAGE')]}),
 ("I'll have a small Cappuccino, please.",
  {'entities': [(18, 27, 'BEVERAGE')]})]

In [ ]:
# Import requirements
import random
from spacy.util import minibatch, compounding
from pathlib import Path
from spacy.training import Example

# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):

  # Training for 30 iterations
  for iteration in range(100):

    # shuufling examples  before every iteration
    random.shuffle(train_set)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = minibatch(train_set, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        example = []
        # Update the model with iterating each text
        for i in range(len(texts)):
            doc = nlp.make_doc(texts[i])
            example.append(Example.from_dict(doc, annotations[i]))
        
        # Update the model
        nlp.update(example, drop=0.5, losses=losses)

        # print("Losses", losses)

/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Could I get a medium Cappuccino with almond milk?" with entities "[(21, 30, 'BEVERAGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "I'll have a small Cappuccino, please." with entities "[(18, 27, 'BEVERAGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "I'd like a medium Cappuccino with a dash of vanill..." with e

In [ ]:
# Testing the model
doc = nlp("May I have a medium Cappuccino with a dollop of whipped cream?")
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities [('Cappuccino', 'BEVERAGE')]


# Adding another Entity (Size)

In [ ]:
# Load pre-existing spacy model
import spacy
nlp=spacy.load('en_core_web_sm')

In [ ]:
# Adding the entity to the ruler before fine tuning
ruler = nlp.add_pipe('entity_ruler')

patterns = [
    {"label":"BEVERAGE","pattern":"cappuccino"},
    
]

ruler.add_patterns(patterns)

In [ ]:
# Testing before fine tuning
sentence = "I'd like a medium Cappuccino with extra foam and a small Latte."
doc = nlp(sentence.lower())
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities [('cappuccino', 'BEVERAGE')]


In [ ]:
# Testing the model
doc = nlp("I'd like a large Cappuccino with extra foam and a Latte.")
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities [('Cappuccino', 'PERSON'), ('Latte', 'ORG')]


We can see that it is incorrectly categorizing "Cappuccino" as a "PERSON"

In [ ]:
# Getting the pipeline component
ner=nlp.get_pipe("ner")

In [ ]:
def generate_entity_output(sentence, word, entity_name):
    start_index = sentence.index(word)
    end_index = start_index + len(word) - 1
    output = {
        "entities": [
            (start_index, end_index, entity_name)
        ]
    }
    return (sentence, output)

In [ ]:
with open('custom_dataset_Cappuccino_Latte.txt', encoding='UTF-8') as file:
    # lines = [line.rstrip() for line in file]
    lines = [line.strip('"').rstrip().rstrip('"').lower() for line in file]

In [ ]:
lines[1]

'can i get a medium cappuccino and a large latte to go?'

In [ ]:
train_set = []

for line in lines:
  train_set.append(generate_entity_output(line,"cappuccino","BEVERAGE"))

In [ ]:
len(train_set)

30

In [ ]:
from spacy import training
# Adding labels to the `ner`

for _, annotations in train_set:
  for ent in annotations.get("entities"):
    ner.add_label(ent[2])

In [ ]:
# Disable pipeline components you dont need to change
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [ ]:
train_set[0:10]

[("i'll have a small cappuccino and a medium latte, please.",
  {'entities': [(18, 27, 'BEVERAGE')]}),
 ('can i get a medium cappuccino and a large latte to go?',
  {'entities': [(19, 28, 'BEVERAGE')]}),
 ("i'd like a large cappuccino and a small latte with an extra shot of espresso.",
  {'entities': [(17, 26, 'BEVERAGE')]}),
 ('may i have a small cappuccino and a medium latte with almond milk, please?',
  {'entities': [(19, 28, 'BEVERAGE')]}),
 ('could i get a medium cappuccino and a large latte with a dusting of cocoa powder?',
  {'entities': [(21, 30, 'BEVERAGE')]}),
 ("i'll take a large cappuccino and a small latte with caramel syrup.",
  {'entities': [(18, 27, 'BEVERAGE')]}),
 ('can i order a small cappuccino and a medium latte with whipped cream?',
  {'entities': [(20, 29, 'BEVERAGE')]}),
 ("i'd like a medium cappuccino and a large latte with soy milk, please.",
  {'entities': [(18, 27, 'BEVERAGE')]}),
 ('may i have a small cappuccino and a medium latte with a sprinkle of cinnamo

In [ ]:
# Import requirements
import random
from spacy.util import minibatch, compounding
from pathlib import Path
from spacy.training import Example

# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):

  # Training for 30 iterations
  for iteration in range(100):

    # shuufling examples  before every iteration
    random.shuffle(train_set)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = minibatch(train_set, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        example = []
        # Update the model with iterating each text
        for i in range(len(texts)):
            doc = nlp.make_doc(texts[i])
            example.append(Example.from_dict(doc, annotations[i]))
        
        # Update the model
        nlp.update(example, drop=0.5, losses=losses)

        # print("Losses", losses)

/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "can i order a small cappuccino and a medium latte ..." with entities "[(20, 29, 'BEVERAGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "may i have a small cappuccino and a medium latte w..." with entities "[(19, 28, 'BEVERAGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "could i get a medium cappuccino and a lar

In [ ]:
# Testing the model
sentence = "May I have a medium Cappuccino and a  Latte with a dollop of whipped cream Amazon?"
doc = nlp(sentence.lower())
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities [('medium', 'SIZE'), ('cappuccino', 'BEVERAGE')]


In [ ]:
# Creating the order list
order = {"Item":[],
         "Size":[]}

input_sentence = ""

while input_sentence != 'confirm':
  input_sentence = input("What would you like to order? : ")
  doc = nlp(input_sentence.lower())

  for i,ent in enumerate(doc.ents):
    if ent.label_ == "BEVERAGE":
      order['Item'].append(ent.text)
      if i!=0 and doc.ents[i-1].label_=='SIZE':
        order['Size'].append(doc.ents[i-1].text)

What would you like to order? : small latte
What would you like to order? : confirm


In [ ]:
order

{'Item': [], 'Size': []}

# Without Pattern (Add Entity)
Only adding the entity to the ruler

In [ ]:
import spacy

In [ ]:
nlp = spacy.load("en_core_web_sm")
ner = nlp.get_pipe('ner')
ner.add_label('BEVERAGE')

1

In [ ]:
# ruler = nlp.add_pipe('entity_ruler')

In [ ]:
# patterns = [
#     {"label":"BEVERAGE","pattern":"cappuccino"}
# ]

In [ ]:
# ruler.add_patterns(patterns)

In [ ]:
# Testing before fine tuning
sentence = "I'd like a medium Cappuccino with extra foam and a small Latte."
doc = nlp(sentence.lower())
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities []


In [ ]:
# Testing the model
doc = nlp("I'd like a large Cappuccino with extra foam and a Latte.")
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities [('Cappuccino', 'PERSON'), ('Latte', 'ORG')]


We can see that it is incorrectly categorizing "Cappuccino" as a "PERSON"

In [ ]:
def generate_entity_output(sentence, word, entity_name):
    start_index = sentence.index(word)
    end_index = start_index + len(word) - 1
    output = {
        "entities": [
            (start_index, end_index, entity_name)
        ]
    }
    return (sentence, output)

In [ ]:
with open('custom_dataset_Cappuccino_Latte.txt', encoding='UTF-8') as file:
    # lines = [line.rstrip() for line in file]
    lines = [line.strip('"').rstrip().rstrip('"').lower() for line in file]

In [ ]:
lines[1]

'can i get a medium cappuccino and a large latte to go?'

In [ ]:
train_set = []

for line in lines:
  train_set.append(generate_entity_output(line,"cappuccino","BEVERAGE"))

In [ ]:
len(train_set)

30

In [ ]:
from spacy import training
# Adding labels to the `ner`

for _, annotations in train_set:
  for ent in annotations.get("entities"):
    ner.add_label(ent[2])

In [ ]:
# Disable pipeline components you dont need to change
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [ ]:
train_set[0:10]

[("i'll have a small cappuccino and a medium latte, please.",
  {'entities': [(18, 27, 'BEVERAGE')]}),
 ('can i get a medium cappuccino and a large latte to go?',
  {'entities': [(19, 28, 'BEVERAGE')]}),
 ("i'd like a large cappuccino and a small latte with an extra shot of espresso.",
  {'entities': [(17, 26, 'BEVERAGE')]}),
 ('may i have a small cappuccino and a medium latte with almond milk, please?',
  {'entities': [(19, 28, 'BEVERAGE')]}),
 ('could i get a medium cappuccino and a large latte with a dusting of cocoa powder?',
  {'entities': [(21, 30, 'BEVERAGE')]}),
 ("i'll take a large cappuccino and a small latte with caramel syrup.",
  {'entities': [(18, 27, 'BEVERAGE')]}),
 ('can i order a small cappuccino and a medium latte with whipped cream?',
  {'entities': [(20, 29, 'BEVERAGE')]}),
 ("i'd like a medium cappuccino and a large latte with soy milk, please.",
  {'entities': [(18, 27, 'BEVERAGE')]}),
 ('may i have a small cappuccino and a medium latte with a sprinkle of cinnamo

In [ ]:
# Import requirements
import random
from spacy.util import minibatch, compounding
from pathlib import Path
from spacy.training import Example

# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):

  # Training for 30 iterations
  for iteration in range(100):

    # shuufling examples  before every iteration
    random.shuffle(train_set)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = minibatch(train_set, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        example = []
        # Update the model with iterating each text
        for i in range(len(texts)):
            doc = nlp.make_doc(texts[i])
            example.append(Example.from_dict(doc, annotations[i]))
        
        # Update the model
        nlp.update(example, drop=0.5, losses=losses)

        # print("Losses", losses)

/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "may i have a small cappuccino and a medium latte w..." with entities "[(19, 28, 'BEVERAGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "could i get a medium cappuccino and a large latte ..." with entities "[(21, 30, 'BEVERAGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "i'll have a small cappuccino and a medium

In [ ]:
# Testing the model
sentence = "May I have a medium Cappuccino and a  Latte with a dollop of whipped cream Amazon?"
doc = nlp(sentence.lower())
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities []


# New way (Custom NER)

In [ ]:
import spacy 
import json
import random

In [ ]:
def train_spacy(TRAIN_DATA,iterations):
  nlp = spacy.blank("en")
  ner = nlp.create_pipe("ner")
  ner.add_label("BEVERAGE")
  nlp.add_pipe(ner, name="beverage_ner")
  other_pipes = [pipe for pipe in nlp.pipe_names if pipe!="beverage_ner"]
  
  with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.begin_training()
    for itn in range(iterations):
        print(f"Starting iteration {str(itn)}")
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in TRAIN_DATA:
          nlp.update([text],
                    [annotations],
                    drop=0.2,
                    sgd=optimizer,
                    losses=losses)
        print(losses)
  return (nlp)


        # random.shuffle(train_set)
        # losses = {}
        # # batch up the examples using spaCy's minibatch
        # batches = minibatch(train_set, size=compounding(4.0, 32.0, 1.001))
        # for batch in batches:
        #     texts, annotations = zip(*batch)
        #     example = []
        #     # Update the model with iterating each text
        #     for i in range(len(texts)):
        #         doc = nlp.make_doc(texts[i])
        #         example.append(Example.from_dict(doc, annotations[i]))
            
        #     # Update the model
        #     nlp.update(example, drop=0.5, losses=losses)



In [ ]:
train_spacy(train_set,10)

ValueError: ignored

In [ ]:
train_set[0]

("i'd like a large cappuccino and a small latte with coconut milk, please.",
 {'entities': [(17, 26, 'BEVERAGE')]})


### Loading Data

In [ ]:
def generate_entity_output(sentence, word, entity_name):
    start_index = sentence.index(word)
    end_index = start_index + len(word) - 1
    output = {
        "entities": [
            (start_index, end_index, entity_name)
        ]
    }
    return (sentence, output)

In [ ]:
with open('custom_dataset_Cappuccino_Latte.txt', encoding='UTF-8') as file:
    # lines = [line.rstrip() for line in file]
    lines = [line.strip('"').rstrip().rstrip('"').lower() for line in file]

In [ ]:
train_set = []

for line in lines:
  train_set.append(generate_entity_output(line,"cappuccino","BEVERAGE"))

In [ ]:
from spacy import training
# Adding labels to the `ner`
nlp = spacy.blank("en")
ner = nlp.create_pipe("ner")

for _, annotations in train_set:
  for ent in annotations.get("entities"):
    print(ent[2])
    ner.add_label(ent[2])

In [ ]:
# Disable pipeline components you dont need to change
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [ ]:
# Import requirements
import random
from spacy.util import minibatch, compounding
from pathlib import Path
from spacy.training import Example

# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):

  # Training for 30 iterations
  for iteration in range(100):

    # shuufling examples  before every iteration
    random.shuffle(train_set)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = minibatch(train_set, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        example = []
        # Update the model with iterating each text
        for i in range(len(texts)):
            doc = nlp.make_doc(texts[i])
            example.append(Example.from_dict(doc, annotations[i]))
        
        # Update the model
        nlp.update(example, drop=0.5, losses=losses)

        # print("Losses", losses)

/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "could i get a medium cappuccino and a large latte ..." with entities "[(21, 30, 'BEVERAGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "i'd like a large cappuccino and a small latte with..." with entities "[(17, 26, 'BEVERAGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "may i have a small cappuccino and a mediu

In [ ]:
# Testing the model
sentence = "May I have a medium Cappuccino and a  Latte with a dollop of whipped cream Amazon?"
doc = nlp(sentence.lower())
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities []
